In [32]:
from data_utils import *
from dataset import TLDataset
from transformers import T5ForConditionalGeneration,T5Tokenizer, AutoConfig
from torch.utils.data import Dataset, DataLoader
from finetuner import T5FineTuner

/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-02-09 10:47:33.853354: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 10:47:34.100746: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-09 10:47:36.941561: W tensorflow/compiler/xla/stream_execut

In [1]:
import torch 
from multiprocessing import cpu_count
import argparse
output_path_dir = "/home/salzubi_umass_edu/CKPTS/OAI_TRG_MODEL_CKPT/"
train_bsz = 4
lr = 1e-3
train_fraction = 1
model_name = "t5-small"
file_name = f"OAI_trg_MODEL_task_Adam_lr:{lr}_bsz:{train_bsz}_frac:{train_fraction}_{model_name}"
num_gpus = torch.cuda.device_count()
num_cpus = cpu_count()

args_dict = dict(
    # file_name = file_name,
    # output_dir=output_path_dir, # path to save the checkpoints
    model_name_or_path=model_name,
    tokenizer_name_or_path=model_name,
    max_seq_length=512,
    train_fraction= train_fraction,
    learning_rate=lr,
    weight_decay=0.0,
    scheduler_factor = .1,
    use_gpu = True if num_gpus > 0 else False,
    train_batch_size=train_bsz,
    eval_batch_size=16,
    num_train_steps=10000000,
    es_patience = 4,
    val_check_interval = .25,
    dropout = .2,
    n_gpu=num_gpus,
    cpu_per_device=1,
    task = "classification",
)
args = argparse.Namespace(**args_dict)

/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [35]:
train, val, test = generate_transfer_splits("gofigure", data_path = "/work/salzubi_umass_edu/human_eval_datasets/master_dataset.csv", random_state=1, train_fraction = args.train_fraction)

In [ ]:
test_model = T5FineTuner(hparams = args).load_from_checkpoint("../experiments/OAI_t5_large_train_fraction=1_TL.ckpt", hparams = args)
print(f"loaded best model...")
test_loader = DataLoader(test_data, batch_size=int(args.eval_bsz), num_workers = num_cpus) #, num_workers=num_cpus//num_gpus

y_preds = []
y_true= []

for idx, batch in enumerate(test_loader):

    y_pred, y = process_test_output(batch, test_model, tokenizer)

    y_preds += y_pred
    y_true += y

final_df = pd.DataFrame({"predicted": y_preds, "ground_truth": y_true})
final_df.to_csv(f"{args.output_dir}{args.file_name}_eval.csv", index=False)
print("finished!!!!")

In [7]:
random_seed = 1
num_gpus = torch.cuda.device_count()
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5FineTuner(args)
train, val, test = generate_transfer_splits("OAI", data_path = "/work/salzubi_umass_edu/human_eval_datasets/master_dataset.csv", random_state=random_seed)
train_data = TLDataset(train, tokenizer)
val_data = TLDataset(val, tokenizer)
test_data = TLDataset(test, tokenizer)
train_sampler = generate_weighted_sampler(train)

train_loader = DataLoader(train_data, batch_size=int(4), drop_last=True, shuffle=False) #sampler = sampler , num_workers=num_cpus//num_gpus
val_loader = DataLoader(val_data, batch_size=int(16)) #, num_workers=num_cpus//num_gpus
test_loader = DataLoader(test_data, batch_size=int(16)) #, num_workers=num_cpus//num_gpus

/work/salzubi_umass_edu/.conda/envs/wmt_env/lib/python3.8/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [9]:
y_preds = []
y_true=
for idx, batch in tqdm(enumerate(test_loader)):
    y_preds += generate_text_batch(batch, model, tokenizer)
    y_true += tokenizer.batch_decode(batch['target_ids'], skip_special_tokens=True)
    

In [ ]:
labels = batch["target_ids"]
labels[labels[:, :] == model.tokenizer.pad_token_id] = -100

In [ ]:
outputs = model(
  input_ids=batch["source_ids"],
  attention_mask=batch["source_mask"],
  labels=labels,
  decoder_attention_mask=batch['target_mask']
)

In [ ]:
from evaluate import generate_text_from_example

generated_text = generate_text_from_example(batch, model, tokenizer)

In [ ]:
generated_text

In [55]:
import pandas as pd

df = pd.read_csv("../human_eval_datasets/master_dataset.csv")
print(len(df))

77095


In [63]:
df[(df.dataset == "mauve") & (df.dataset_type == "SRC")].labels.value_counts(normalize=True)

A    0.602778
B    0.397222
Name: labels, dtype: float64

In [48]:
test_split = df[(df.dataset_type == "TRG") & (df.dataset == "OAI")]
train_with = test_split.sample(int(.5 * len(test_split)))
test_split = test_split[~test_split.index.isin(train_with.index)]

In [51]:
df[df.dataset_type == "SRC"]

,input_text,labels,prefix,dataset,dataset_type
0,Answer the following question given the contex...,"It stated, that Dark Castle was ""the best arca...",question answering,OAI,SRC
1,Answer the following question given the contex...,Fox,question answering,OAI,SRC
2,Answer the following question given the contex...,"The team is comprised of doctors, nurses, phar...",question answering,OAI,SRC
3,Answer the following question given the contex...,Fungi thrive in these areas.,question answering,OAI,SRC
5,Answer the following question given the contex...,It might be implied it's not advisable because...,question answering,OAI,SRC
...,...,...,...,...,...
77086,Answer the following question given the contex...,He might raise it as his own.,question answering,OAI,SRC
77090,Answer the following question given the contex...,a historic home,question answering,OAI,SRC
77091,Answer the following question given the contex...,After the ruling Puea Thai party tried to intr...,question answering,OAI,SRC
77093,Which continuation is more likely to be writte...,B,binary classification,mauve,SRC


In [1]:
import pandas as pd

df = pd.read_csv("../human_eval_datasets/master_dataset.csv")

In [2]:
df.head()

,input_text,labels,prefix,dataset,dataset_type,task_id
0,"Given the following question, which response i...",A,binary classification,shp,SRC,64
1,Given the following English to Chinese transla...,minor,multiclass classification,wmt_zh_en,SRC,16
2,Given the following English to German translat...,no-error,multiclass classification,wmt_en_de,SRC,14
3,"Given the following question, which response i...",A,binary classification,shp,SRC,64
4,"On a scale from 0-5, how similar are the two s...",0.800000011920929,regression,sts_b,SRC,55


In [3]:
target_dataset = df[df.task_id==50]

In [5]:
target_dataset.to_csv("oai_data.csv", index=False)